In [2]:
import tensorflow as tf
import keras
from keras.src.layers import Embedding
from keras.layers import Dense
from keras.src.utils import to_categorical
from keras.models import Sequential

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import pandas

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier 


# Load Data

In [81]:
train_r1 = pd.read_csv('data_mod/Train/HR_R1_Train.csv')
test_r1 = pd.read_csv('data_mod/Test/HR_R1_Test.csv')
interleaved_r1 = pd.read_csv("data_mod/Interleaved/HR_IL_R1.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data_mod/Train/HR_R1_Train.csv'

In [86]:
t1_train = pd.read_csv("data/TRAIN/R1/R1.csv")
t1_test = pd.read_csv("data/TEST/R1/R1.csv")
t1_valid = pd.read_csv("data/VALID/R1/R1.csv")

In [75]:
train_r4 = pd.read_csv('data/TRAIN/R4/R4.csv')
test_r4 = pd.read_csv('data/TEST/R4/R4.csv')
interleaved_r4 = pd.read_csv("data_mod/HR_IL_R4_labeled.csv")

## Potential Features
- *MessageType_WithRole* 
- *Message Attributes*
- *HighestLayerProtocol*

In [87]:
t1_valid

,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,MessageType_WithRole,MessageType,MessageAttributes
0,0,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.599660,48,2020-12-21 21:40:04.619660,2020-12-21 21:40:04.619660,192.168.11.1-192.168.11.2 (1),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':..."
1,1,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.600052,49,2020-12-21 21:40:08.540052,2020-12-21 21:40:08.540052,192.168.11.1-192.168.11.2 (1),pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Parameter status,"{'pgsql.type': 'Parameter status', 'pgsql.leng..."
2,2,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.600052,49,2020-12-21 21:40:08.540052,2020-12-21 21:40:08.540052,192.168.11.1-192.168.11.2 (1),pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Parameter status,"{'pgsql.type': 'Parameter status', 'pgsql.leng..."
3,3,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.600052,49,2020-12-21 21:40:08.540052,2020-12-21 21:40:08.540052,192.168.11.1-192.168.11.2 (1),pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Command completion,"{'pgsql.type': 'Command completion', 'pgsql.le..."
4,4,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.600052,49,2020-12-21 21:40:08.540052,2020-12-21 21:40:08.540052,192.168.11.1-192.168.11.2 (1),pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Ready for query,"{'pgsql.type': 'Ready for query', 'pgsql.lengt..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223978,3223978,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.704720,4993,2021-01-10 13:47:18.324720,2021-01-10 13:47:18.324720,192.168.11.1-192.168.11.2 (7),pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Ready for query,"{'pgsql.type': 'Ready for query', 'pgsql.lengt..."
3223979,3223979,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.704720,4993,2021-01-10 13:47:18.324720,2021-01-10 13:47:18.324720,192.168.11.1-192.168.11.2 (7),pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Command completion,"{'pgsql.type': 'Command completion', 'pgsql.le..."
3223980,3223980,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.704810,4994,2021-01-10 13:47:19.224810,2021-01-10 13:47:19.224810,192.168.11.1-192.168.11.2 (3),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':..."
3223981,3223981,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.705430,4995,2021-01-10 13:47:25.425430,2021-01-10 13:47:25.425430,192.168.11.1-192.168.11.2 (3),pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Command completion,"{'pgsql.type': 'Command completion', 'pgsql.le..."


# Generate Labels
Add activity_start, activity_end and nothing type as labels

In [18]:
# Mark start event of each BusinessActivity Instance
train_r1["activityStart"] = train_r1.groupby(["BusinessActivity","InstanceNumber",]).cumcount()==0
# Mark end event of each Business Activity Instance
train_r1["activityEnd"] = train_r1.groupby(["BusinessActivity","InstanceNumber",]).cumcount(ascending=False)==0
# Merge start and end columns to form labels
train_r1["task_position"] = train_r1.apply(lambda row: "position_start" if row["activityStart"] else ("position_end" if row["activityEnd"] else 'position_between'), axis=1)

train_r1 = train_r1.drop(["activityStart",'activityEnd'], axis=1)

# Preprocess Features

In [19]:
print(train_r1["MessageType"].value_counts())
print(len(train_r1["MessageType"].value_counts()))
display(train_r1[train_r1["task_position"]!="position_between"]["MessageType"].value_counts())

MessageType
PgsqlResponse:Data row                            622758
PgsqlResponse:Ready for query                     221796
PgsqlResponse:Command completion                  216013
PgsqlRequest:Simple query                         216002
PgsqlResponse:Row description                     123654
PgsqlResponse:Parameter status                    104063
Connection finish (FIN)                            13107
Connection establish acknowledge (SYN+ACK)         11821
Connection establish request (SYN)                 11821
PgsqlResponse:Authentication request               11567
PgsqlRequest:Startup message                        5784
PgsqlRequest:SSL request                            5784
PgsqlRequest:Password message                       5784
PgsqlResponse:Backend key data                      5783
PgsqlRequest:Termination                            5753
HttpResponse:HTTP/1.0 200 OK\r\n                    5346
SmtpDataFragment                                    3844
HttpRequest:POST /x

MessageType
Connection establish request (SYN)    653
Connection finish (FIN)               516
PgsqlResponse:Ready for query         415
PgsqlRequest:Simple query             279
PgsqlResponse:Error                     1
PgsqlResponse:Row description           1
PgsqlResponse:Parameter status          1
Name: count, dtype: int64

In [27]:
# Split Columns
train_r1[["session","session_id"]] = train_r1["session_generalized"].str.split(" ", n=1, expand=True)
train_r1["session_id"] = train_r1["session_id"].str.replace('(','').str.replace(')','')
train_r1.drop("session_generalized",axis=1,inplace=True)

Message types overlap between normal elements and activity end events 
-> In contrast to UI-Logs where we have Ok, Next, Submit Actions, etc. there is not one clear type.
-> Try to train sequences of message type

In [30]:
train_r1

,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,synthetic_sniff_time,synthetic_sniff_time_str,HighestLayerProtocol,MessageType_WithRole,MessageType,MessageAttributes,task_position,session,session_id
0,0,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,1,2020-05-13 10:07:13.772278,2020-05-13 10:07:13.772278,tcp,End Point (HR Manager)->Odoo Application:[Conn...,Connection establish request (SYN),{},position_start,192.168.11.2-192.168.11.6,1
1,1,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,1,2020-05-13 10:07:15.732474,2020-05-13 10:07:15.732474,tcp,Odoo Application->End Point (HR Manager):[Conn...,Connection establish acknowledge (SYN+ACK),{},position_between,192.168.11.2-192.168.11.6,1
2,2,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,1,2020-05-13 10:07:18.092710,2020-05-13 10:07:18.092710,http,End Point (HR Manager)->Odoo Application:[Http...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\\r\\n', '...",position_between,192.168.11.2-192.168.11.6,1
3,3,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,1,2020-05-13 16:14:45.937274,2020-05-13 16:14:45.937274,tcp,Odoo Application->db Server/Mail Server:[Conne...,Connection establish request (SYN),{},position_between,192.168.11.1-192.168.11.2,1
4,4,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,1,2020-05-13 16:14:47.787459,2020-05-13 16:14:47.787459,tcp,db Server/Mail Server->Odoo Application:[Conne...,Connection establish acknowledge (SYN+ACK),{},position_between,192.168.11.1-192.168.11.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606177,1606177,ContractProposal_94.pcap,ContractProposal,94,2020-05-14 21:07:18.607078,2020-05-14 21:07:18.607078,pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Command completion,"{'pgsql.type': 'Command completion', 'pgsql.le...",position_between,192.168.11.1-192.168.11.2,5
1606178,1606178,ContractProposal_94.pcap,ContractProposal,94,2020-05-14 21:07:18.607078,2020-05-14 21:07:18.607078,pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Parameter status,"{'pgsql.type': 'Parameter status', 'pgsql.leng...",position_between,192.168.11.1-192.168.11.2,5
1606179,1606179,ContractProposal_94.pcap,ContractProposal,94,2020-05-14 21:07:18.607078,2020-05-14 21:07:18.607078,pgsql,db Server/Mail Server->Odoo Application:[Pgsql...,PgsqlResponse:Ready for query,"{'pgsql.type': 'Ready for query', 'pgsql.lengt...",position_between,192.168.11.1-192.168.11.2,5
1606180,1606180,ContractProposal_94.pcap,ContractProposal,94,2020-05-14 21:07:26.607878,2020-05-14 21:07:26.607878,http,Odoo Application->End Point (HR Manager):[Http...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\\r\\n', '_ws.expert': 'E...",position_between,192.168.11.2-192.168.11.6,9


## Approach 1 - 
- Form Sequences or Length N
- Tokenize Messagetype 
- Train Model


### Encoding

In [31]:
# Label Encode the Message Type
messageTypeEncoder = LabelEncoder()
messageTypeEncoder.fit(train_r1["MessageType"].values)
# Shift the labels by 1 to exclude 0 as label (we use it for padding starting sequences)
train_r1["MessageType_labels"] = messageTypeEncoder.transform(train_r1["MessageType"]) + 1

In [32]:
sessionEncoder = LabelEncoder()
sessionEncoder.fit(train_r1["session"].values)
train_r1["session_labels"] = sessionEncoder.transform(train_r1["session"])+1

In [21]:
# One hot encode the Labels 
encoder = LabelEncoder()
y = encoder.fit_transform(train_r1["task_position"])
dummy_y = to_categorical(y)

### Form Sequences of Length N

In [55]:
from numpy.lib.stride_tricks import sliding_window_view

# Todo build custom sliding window function forward + backward facing w. custom windowsize
sequence_length = 20
#labels = train_r1["MessageType_labels"].rolling(window=windowsize).agg(list)
sequences = sliding_window_view(train_r1["MessageType_labels"], sequence_length)

# pad first n-1 elements
pad_top = [[0] * (sequence_length - 1 - i) + sequences[0, 0:i + 1].tolist() for i in range(sequence_length - 1)]

X = np.insert(sequences,0, pad_top, axis=0).tolist()
X = np.asarray(X).astype(np.float32)

In [61]:
session_sequences = sliding_window_view(train_r1["session_labels"], sequence_length)
# pad first n-1 elements
pad_top = [[0] * (sequence_length - 1 - i) + session_sequences[0, 0:i + 1].tolist() for i in range(sequence_length - 1)]

X_1 = np.insert(session_sequences,0, pad_top, axis=0).tolist()
X_1 = np.asarray(X).astype(np.float32)

In [23]:
from keras.src.layers import LSTM


def baseline_model():
   
    return model

In [32]:
from keras.src.layers import LSTM
from keras import Input


def baseline_model():
    embedding_dim = 150

    vocab_size = len(messageTypeEncoder.classes_) + 1
    model = Sequential([
        Input(shape=(sequence_length,)),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        LSTM(units=64),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def stacked_LSTM(): 
    embedding_dim = 150
    vocab_size = len(messageTypeEncoder.classes_) +1
    model = Sequential([
        Input(shape= (sequence_length,)),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        LSTM(64, activation='relu', return_sequences=True),
        LSTM(64, activation='relu'),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['f1_score']) 
    return model

def stacked_LSTM_weighted():
    embedding_dim = 150
    vocab_size = len(messageTypeEncoder.classes_) +1
    model = Sequential([
        Input(shape= (sequence_length,)),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        LSTM(64, activation='relu', return_sequences=True),
        LSTM(64, activation='relu'),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['f1_score']) 
    return model


    



In [29]:
x_train,x_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.2, shuffle=True, random_state=42)

In [33]:
estimator = KerasClassifier(model=stacked_LSTM_weighted(), epochs=5, batch_size=1000, verbose=1)
estimator.fit(x_train, y_train)
#kfold = KFold(n_splits=1, shuffle=True)
#results = cross_val_score(estimator, X, dummy_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
  25/1285 ━━━━━━━━━━━━━━━━━━━━ 20:12 962ms/step - f1_score: 0.3115 - loss: 0.8088

KeyboardInterrupt: 